In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget 

import pickle
import os
import sys
import torch
import math
import random
from typing import List, Tuple, Dict
import json
import itertools
from pathlib import Path
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import trange, tqdm
from filelock import FileLock
import tap
from natsort import natsorted
import numpy as np
import torch.utils.data as data
from torch.nn import functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as transforms_f
from rlbench.observation_config import ObservationConfig, CameraConfig
from rlbench.environment import Environment
from rlbench.backend.observation import Observation
from rlbench.task_environment import TaskEnvironment
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.action_modes.arm_action_modes import EndEffectorPoseViaPlanning
from rlbench.backend.exceptions import InvalidActionError
from PIL import Image
from create_data import RLBench
import ipdb
import open3d as o3d
from open3d.web_visualizer import draw
import pickle
import numpy as np
import torch
from scipy.spatial.transform import Rotation as R
from rotations import compute_rotation_matrix_from_quaternions
from network import PlainUNet, TransformerUNet
from utils import (
    RLBenchEnv,
    task_file_to_task_class,
    count_parameters,
    load_episodes,
    load_instructions
)
from data_gen_sim2real import Arguments as S2RArguments, load_demo, Demo, get_obs_action_from_demo
from utils_sim2real import get_gripper_state, depth_to_pcd, get_extrinsics, get_intrinsics, square, get_rgb_pcd_gripper_from_obs,get_extrinsics,get_intrinsics


task = 'put_money_in_safe'
sys.argv = ['foo']

In [ ]:
dataset = RLBench('dataset', task)
frame_id, state, action, padding_mask = dataset[5]
print(frame_id, state.shape, action.shape)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=dataset, batch_size=32//5, shuffle=True, num_workers=0
)
train_dataset = iter(train_loader)
device = 'cpu'

for i in trange(8000):
    try:
        frame_id, state, action, padding_mask = train_dataset.next()
    except:
        train_dataset = iter(train_loader)
        frame_id, state, action, padding_mask = train_dataset.next()

    inputs = state.to(device)
    outputs = action.to(device)

    rgb, pc = torch.split(
        inputs, [inputs.shape[2] // 2, inputs.shape[2] // 2], dim=2
    )
    
    break

In [ ]:
tasks = [task]
episodes = load_episodes()
max_eps_list = {task_str: episodes[task_str] for task_str in tasks}

t_list: Dict[str, torch.Tensor] = {}
z_list: Dict[str, torch.Tensor] = {}
for task_str, max_eps in max_eps_list.items():
    t_list[task_str] = torch.rand(max_eps, 64, requires_grad=True, device=device)
    z_list[task_str] = torch.zeros(max_eps, 3, requires_grad=True, device=device)

# model = PlainUNet(action_size=8, depth=4, temp_len=64).to(device)
model = TransformerUNet(action_size=8, depth=4, temp_len=64).to(device)

# Visualization

### Raw RLBench sample

In [ ]:
from utils import RLBenchEnv
from data_gen import Arguments as DataArguments
args = DataArguments().parse_args()
args.data_dir = Path('real_raw')
env = RLBenchEnv(
        data_path=args.data_dir,
        apply_rgb=True,
        apply_pc=True,
        apply_cameras=args.cameras,
    )

In [ ]:
tasks = [
    #"stack_wine", "stack_blocks", "sweep_to_dustpan", 
    # "pick_up_cup",
    #"toilet_seat_down", "close_fridge", 
    #"/scratch/e_usb_out_of_computer", 
    # "water_plants",
    #"change_clock", 
    #"insert_into_square_peg",
    #"take_lid_off_saucepan"
    "push_buttons"
]

for task in tasks:
    demo = env.get_demo(task, 4, i)
    key_frame = keypoint_discovery(demo)
    print(task, key_frame)
    #key_frame = [k for i, k in enumerate(key_frame) if i % 6 in set([1, 4])]
    #for i, kp in enumerate(key_frame):
    #    plt.subplot(131)
    #    plt.imshow(demo[kp].left_shoulder_rgb)
    #   plt.subplot(132)
    #    plt.imshow(demo[kp].right_shoulder_rgb)
    #    plt.subplot(133)
    #    plt.imshow(demo[kp].wrist_rgb)
    #    plt.title(f'{kp} {i}')
    #    plt.show()

In [ ]:
tasks = [
     "insert_onto_square_peg",
    "stack_wine", 
    "stack_blocks", "sweep_to_dustpan", 
     "pick_up_cup",
    "toilet_seat_down", "close_fridge", 
    "take_usb_out_of_computer", 
     "water_plants",
    "change_clock", 
   #
    "take_lid_off_saucepan"
]
root_dir =Path( "/scratch/diffrac/datasets/vln-robot/")
dest_dir=Path("/home/pguhur/teaser_fig")
dest_dir.mkdir(exist_ok=True)
variation = 0
for task in tasks:
    for episode in range(5):
        front_rgb_dir = root_dir / f"{task}/variation{variation}/episodes/episode{episode}/front_rgb/"
        if front_rgb_dir.is_dir():
            break
    assert front_rgb_dir.is_dir(),front_rgb_dir
    front_rgbs = front_rgb_dir.glob("*.png")
    front_rgbs = natsorted(front_rgbs)
    
    front_mask_dir = root_dir / f"{task}/variation{variation}/episodes/episode{episode}/front_mask/"
    front_masks = front_mask_dir.glob("*.png")
    front_masks = natsorted(front_masks)
    
    print(task, len(front_rgbs))
    
    for front_rgb in [front_rgbs[0]]:
        # , front_rgbs[-1]]:
        fid = int(front_rgb.stem)

        img = Image.open(front_rgb)
        timg = torch.from_numpy(np.asarray(img))

        mask = Image.open(front_mask_dir/f"{fid}.png")
        tmask = torch.from_numpy(np.asarray(mask))

        objects = tmask.view(-1, 3).unique(dim=0)
        # print(objects)
        robot_mask = torch.zeros_like(tmask[:,:,0]).bool()
        for i in range(31,47):
            part = torch.Tensor([i, 0, 0]).view(1,1,3)
            robot_mask |= (tmask == part).all(dim=2)

        alpha_mask = Image.fromarray(robot_mask.int().numpy()*255).convert('L')
        img.putalpha(alpha_mask)
        img.save(dest_dir / f"{task}-{variation}-{episode}-{fid}-mask.png")
        
    img = Image.open(front_rgbs[-1])
    img.save(dest_dir / f"{task}-{variation}-{episode}-{fid}-img.png")

### Packaged sample

In [ ]:
import itertools
import matplotlib.pyplot as plt
import torch
from utils import (
    load_instructions
)
from create_data import RLBench

# root_dir = "/scratch/jeanzay/scratch/datasets/vln-robot/alhistory/datasets/real-dataset-0/"
# root_dir = "datasets/real-dataset-0"
root_dir = "datasets/"

taskvar = list(itertools.product(['tower3'], [0]))
instruction = load_instructions("instructions-clip.pkl")
max_episode_length = 2
dataset = RLBench(
    root=root_dir,
    taskvar=taskvar,
    instructions=instruction,
    gripper_pose='none',
    taskvar_token=False,
    max_episode_length=max_episode_length,
    max_episodes_per_taskvar=100,
    cache_size=100,
    num_iters=100,
    jitter=True,
    training=True,
)

In [ ]:
sample = dataset[0]
rgb = (sample['rgbs'] + 1) * 0.5
rgb = rgb.permute(0, 1, 3, 4, 2)

for i in range(rgb.shape[0]):
    plt.imshow(rgb[i, 0])
    plt.title(("{:.2f} "*8).format(*sample['action'][i, :8].tolist()))
    plt.show()

### Raw Sim2Real sample

In [ ]:
ep_dir = Path(f'raw-dr/variation0/episodes/episode0/')
demo = load_demo(ep_dir)

In [ ]:
%matplotlib inline 
for i, f in enumerate(demo):
    if f.left_shoulder_rgb is None:
        continue
        
    plt.subplot(121)
    plt.imshow(getattr(f, 'left_shoulder_rgb'))
    plt.subplot(122)
    plt.imshow(getattr(f, 'right_shoulder_rgb'))
    
    plt.title(f"{i}: {f.gripper_open}")
    plt.show()

In [ ]:
idx = 563
pcd = demo[idx].left_shoulder_pcd
rgb = demo[idx].left_shoulder_rgb

# pcd = pcd * (1 + 0.05*torch.empty_like(pcd).normal_())
print(pcd.min(), pcd.max(), rgb.min(), rgb.max())

mesh = o3d.geometry.PointCloud()
mesh.points = o3d.utility.Vector3dVector(pcd.reshape(-1,3))
mesh.colors = o3d.utility.Vector3dVector((rgb/255).reshape(-1,3))

target_pos = demo[idx].gripper_pose[:3]
radius = np.array([0.03]*3)
target = o3d.geometry.AxisAlignedBoundingBox(target_pos - radius, target_pos + radius)

draw([mesh, target])

##### Debug sim2real dataset collection

In [ ]:
from collect_sim2real import collect_episode
import gym
env = gym.make('Var0-Stack-v0')
episode = collect_episode(env)

In [ ]:
%matplotlib inline
for frame in episode.buffer[-10:]:
    plt.imshow(frame['rgb_charlie_camera'])
    plt.show()

In [ ]:
plt.clf()
joint_vel = [
    np.max(frame['arms_joint_vel']) for frame in episode.buffer
]
plt.semilogy(joint_vel, '+')
plt.show()

# Predicting a position without PCD

In [ ]:
# load predicted attn maps + samples
obs = torch.load("obs.pth")
output = torch.load("output.pth")
rgb, pcd, gripper = get_rgb_pcd_gripper_from_obs(obs, "none")
rgb = rgb.permute(0, 1, 3, 4, 2)
rgb = (rgb + 1) * 0.5
pcd = pcd.permute(0, 1, 3, 4, 2)

In [ ]:
for i, im in enumerate(rgb[0]):
    plt.imshow(im.detach().cpu())
    plt.title(i)
    plt.show()

In [ ]:
for i, attn in enumerate(output['attention'][0]):
    plt.imshow(attn[0].detach().cpu().log())
    plt.title(i)
    plt.show()


In [ ]:
fov = obs['info_charlie_camera']['fovy']
attn = output['attention'][0, 2, 0]
h, w = attn.shape

xyz1 = obs['info_charlie_camera']['pos']
euler1 = obs['info_charlie_camera']['euler']
cam1_ext = get_extrinsics(xyz1, euler1)
cam1_int = get_intrinsics(fov, h, w)

xyz2 = np.array([0,0,0])
euler2 = np.array([0,-math.pi/2,0])
cam2_ext = get_extrinsics(xyz2, euler2)
cam2_int = get_intrinsics(fov, h, w)

# pixel coord of 4 corners

In [ ]:
#pts_src = np.array(
#   [
#        [0, 0],
 #       [0, 127],
  #      [127, 127],
   #     [127, 0],
   # ]
#)

In [ ]:
pts_src = np.array(
    [
        [80, 10],
        [80, 100],
        [25, 120],
        [25, 40],
    ]
)

word_coords = np.stack([
    pcd[0, 1, i,j] for i, j in pts_src
])
print(word_coords)

In [ ]:
words_coords = np.array(
[[-0.65052414,  0.22975962,  0.00319054],
 [-0.6482557 , -0.14094314,  0.00319825],
 [-0.28 , -0.30831015,  0.12282395],
 [ -0.28,  0.20341617,  0.00665894]]
)

from utils_sim2real import world_coord_to_pixel
fov = obs['info_charlie_camera']['fovy']
attn = output['attention'][0, 2, 0]
h, w = attn.shape
xyz1 = obs['info_charlie_camera']['pos']
euler1 = obs['info_charlie_camera']['euler']
cam1_ext = get_extrinsics(xyz1, -euler1)
#print("extrinsics")
#print(cam2_ext)
cam1_int = get_intrinsics(fov, h, w)
u,v,depth = world_coord_to_pixel(word_coords, cam1_ext, cam1_int, width=128)
pts_src = np.stack([u, v], 1)
print(u, v, depth)
#pts_src[:, [1, 0]] = pts_src[:, [0, 1]]

xyz2 = np.array([-0.6,0,0.7])
euler2 = np.array([0,0,math.pi/2])
cam2_ext = get_extrinsics(xyz2, -euler2)
print("extrinsics")
print(cam2_ext)
cam2_int = get_intrinsics(fov, h, w)

pts_dst = world_coord_to_pixel(word_coords, cam2_ext, cam2_int, width=128)
pts_dst = np.stack(pts_dst, 1)
print(pts_dst)
#pts_dst[:, [1, 0]] = pts_dst[:, [0, 1]]
print("pts dest")
print(pts_dst)


import cv2
homography, status = cv2.findHomography(pts_src, pts_dst[:, :2])
im_src = rgb[0][1].numpy()
im_out = cv2.warpPerspective(im_src, homography, (im_src.shape[1],im_src.shape[0]))
attn_log_out = cv2.warpPerspective(attn.log().detach().cpu().numpy(), homography, (im_src.shape[1],im_src.shape[0]))

attn_out = cv2.warpPerspective(attn.detach().cpu().numpy(), homography, (im_src.shape[1],im_src.shape[0]))
attn_out = torch.from_numpy(attn_out)

plt.imshow(im_out)
for pt in pts_src:
    plt.scatter(pt[0], pt[1])
plt.show()

plt.imshow(attn_log_out)
plt.colorbar()
plt.show()

In [ ]:
attn_out.max()

In [ ]:
#attn_out = torch.from_numpy(attn_out)
from utils_sim2real import pixel_to_world_coord
print(torch.max(attn_out))
pos_ij = (attn_out==torch.max(attn_out)).nonzero()[0].tolist()
pos_ijd = np.array([[*pos_ij, xyz2[2]]])
print(pos_ijd)
pixel_to_world_coord(pos_ijd, cam2_ext, cam2_int)

# attn mask

In [ ]:
from torch.distributions import Bernoulli
p = 0.5
dist = Bernoulli(torch.ones((B, T)) * p)
diag = dist.sample()
print(diag)
torch.diag_embed(diag).shape

In [ ]:
import einops
T = 6
N = 3
H = 8 
W = 8
B = 2
a = torch.triu(torch.ones((T, T)), diagonal=1)
attn = einops.repeat(a, 't tp -> b t k (tp kp)',  b=B, k=N, kp=N)

dist = Bernoulli(torch.ones(B * T) * p)
diag = dist.sample() .view(B, T)
diag[:, 0] = 0
mask = einops.repeat(torch.diag_embed(diag), 'b t tp -> b t k (tp kp)',  k=N, kp=N)

# mask.long()
print(diag)
attn = (attn.bool() | mask.bool()).long()
attn

## projecting the grippper position as an attn map

In [ ]:
## Load a demo

env = RLBenchEnv(data_path="demos", apply_rgb=True)
task_str = 'stack_wine'
task = task_file_to_task_class(task_str)
demo = env.get_demo(task_str, 0, 0)[0]

In [ ]:
state_ls = torch.load('state.pth')

In [ ]:
for step in state_ls:
    for cam in step:    
        plt.imshow(cam[0].permute(1,2,0))
        plt.show()
        plt.imshow(cam[1].permute(1,2,0))
        plt.show()
    break

In [ ]:
state_ls2 = einops.rearrange(
    state3,
    "(m cam ch) h w -> cam m h w ch", ch=3, cam=3,m=2, 
)
for cam in state_ls2:
    # for cam in step:    
        plt.imshow(cam[0])
        plt.show()
        plt.imshow(cam[1])
        plt.show()

In [ ]:
def obs_to_attn(obs) -> torch.Tensor:
    extrinsics_44 = torch.from_numpy(obs.misc[f'{camera}_camera_extrinsics']).float()
    extrinsics_44 = torch.linalg.inv(extrinsics_44)
    intrinsics_33 = torch.from_numpy(obs.misc[f'{camera}_camera_intrinsics']).float()
    intrinsics_34 = F.pad(intrinsics_33, (0, 1, 0, 0))
    gripper_pos_3 = torch.from_numpy(obs.gripper_pose[:3]).float()
    gripper_pos_41 = F.pad(gripper_pos_3, (0, 1), value=1).unsqueeze(1)
    points_cam_41 = extrinsics_44 @ gripper_pos_41

    proj_31 = intrinsics_34 @ points_cam_41
    proj_3 = proj_31.float().squeeze(1)
    u = (proj_3[0] / proj_3[2]).round().long()
    v = (proj_3[1] / proj_3[2]).round().long()

    attn = torch.zeros((128, 128))
    if not (u < 0 or u > 127 or v < 0 or v > 127):
        attn[v, u] = 1
        
    return attn, (u, v)

for frame_id in keypoint_discovery(demo):
    for camera in ('wrist', 'left_shoulder', 'right_shoulder'):
        obs = demo[frame_id]
        attn, (u, v) = obs_to_attn(obs)
        
        plt.subplot(1, 2, 1)
        plt.imshow(getattr(obs, f'{camera}_rgb'))
        plt.scatter(u, v, s=50, c='green', marker='o')
        plt.plot([64, 64], [0, 128], c='blue')
        plt.plot([0,128],[64, 64], c='blue')
        
        plt.subplot(1, 2, 2)
        plt.imshow(attn)
        
        plt.show()

# Sim2Real

In [ ]:

frames = []  
#     with open(f'sim2real/raw/seed0/tower/variation0/episodes/0000_{frame:05}.pkl', 'rb') as fid:
ep_id = 1
# ep_dir = Path(f'raw-dr/variation0/episodes/')
ep_dir = Path(f'/scratch/azimov/pguhur/datasets/tower/variation5/episodes/')
files = list(sorted(ep_dir.glob(f'{ep_id:04}_*.pkl')))
for f in tqdm(files):
    with open(f, 'rb') as fid:
        data = pickle.load(fid)
    frames.append(data)

In [ ]:
import glob
from PIL import Image
import matplotlib.pyplot as plt

imgs = (Image.fromarray(f['rgb_charlie'][:, 280:-280]) for f in frames)
img = next(imgs)  # extract first image from iterator
img.save(fp=f'{ep_id}.gif', format='GIF', append_images=imgs,
         save_all=True, duration=5, loop=0)

In [ ]:
# world_coord2 = pixel_to_world_coord(np.array([pixel_coord]), extrinsics, intrinsics)
world_coord = data[0]['cube1_pos'][None, :]
print(world_coord)
u, v, depth = world_coord_to_pixel(world_coord, extrinsics, intrinsics, width)
print(u, v, depth)

# pixel to pos2d
pos_2d = np.array([[
    (width - u)*depth,
    v*depth,
    depth
]])
print(transformed_coords_vectopostgres.confr)

rgb = data[1]['rgb_charlie_camera'][:, 280:-280]
plt.imshow(rgb)
plt.scatter(u, v, s=100, c='y', marker='o')
plt.scatter(u, v, s=20, c='r', marker='o')
plt.show()

In [ ]:
# simulation
rgb = data[0]['rgb_charlie_camera0']# [:,280:-280]
depth = data[0]['depth_charlie_camera0']#[:,280:-280]
# real
# rgb = data[1]['rgb_charlie_camera']#
# depth = data[1]['depth_charlie_camera']#[:,280:-280]


fov = float(data[0]['info_charlie_camera']['fovy'])
euler = torch.Tensor(data[0]['info_charlie_camera']['euler'])
pos = torch.Tensor(data[0]['info_charlie_camera']['pos'])
extrinsics = get_extrinsics(pos, euler)

depth2 = depth_resize(depth, extrinsics, fov, 128, 128)
rgb = square(rgb)
rgb2 = rgb_resize(rgb, 128, 128)/255

h2, w2 = depth2.shape
intrinsics2 = get_intrinsics(fov, h2, w2)
pcd2 = depth_to_pcd(depth2, extrinsics, intrinsics2)
mesh = o3d.geometry.PointCloud()
mesh.points = o3d.utility.Vector3dVector(pcd2.reshape(h2*w2,3))
mesh.colors = o3d.utility.Vector3dVector(rgb2.reshape(h2*w2, 3))
points = mesh.get_axis_aligned_bounding_box().get_box_points()
print('Array', np.asarray(points))
bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(points)

# target_pos = data[0]['gripper_pos']
target_pos = data[0]['cube0_pos']
radius = np.array([0.03]*3)
target = o3d.geometry.AxisAlignedBoundingBox(target_pos - radius, target_pos + radius)

draw([mesh, bbox, target], show_ui=True)

In [ ]:
import gym
import muse.envs
from tqdm import trange

seeds = set([])
for var in trange(200):
    env = gym.make(f"Var{var}-PushButtons-v0")
    seeds |= set(env.seed(i) for i in range(2000))
print(len(seeds))

## Visualize samples from Sim2Real env

# Tower

In [ ]:
data_path = Path('/scratch/cooper/datasets/vln-robot/')
env = RLBenchEnv(
    data_path=data_path,
    apply_rgb=True,
    apply_pc=True,
    apply_cameras=('front'),
)

task_str = 'tower3'
task_type = task_file_to_task_class(task_str)
# task = env.get_task(task_type)

In [ ]:
task_inst = env.env.get_task(task_type)._task
task_inst.init_task()

In [ ]:
instr = task_inst.init_episode(1)

In [ ]:
instr

In [ ]:
demos = env.get_demo(task_str, 0, 0)
demo = demos[0]

In [ ]:
frames = keypoint_discovery(demo)

In [ ]:
len(frames)

In [ ]:
demo[f].__dict__.keys()

In [ ]:
demo[f].right_shoulder_rgb

In [ ]:
for f in frames:
    img = Image.open(data_path / task_str / 'variation0' / 'episodes' / 'episode0' / 'front_rgb' / f'{f}.png')
    plt.imshow(img)
    plt.title(f)
    plt.show()

# RLBench datasets

In [ ]:
variation = 81
data = np.load(f'/scratch/jeanzay/scratch/datasets/vln-robot/alhistory/dataset-{variation}/tower+0/ep1.npy', allow_pickle=True)

plt.imshow(data[1][0][0,0].permute(1,2,0))
# plt.imshow()

# Variations on RLBench

In [ ]:
action_mode = MoveArmThenGripper(
    arm_action_mode=EndEffectorPoseViaPlanning(),
    gripper_action_mode=Discrete(),
)
obs_config = ObservationConfig()
env = Environment(
    action_mode, 'datasets', obs_config, headless=True,
)

In [ ]:
task_str = 'push_buttons'
task_type = task_file_to_task_class(task_str)
task = env.get_task(task_type)

In [ ]:
print(task._task.sequences[:5])

In [ ]:
with open(f'{task_str}.json', 'w') as fid:
    json.dump(task._task.sequences[:300], fid, indent=2)

In [ ]:
sequences= task._task.sequences[:200]

In [ ]:
with open('push_buttons.json') as fid:
    sequences = json.load(fid)

In [ ]:
our_colors = set(
    ["yellow", "green", "cyan",
     "blue", "white", "purple",
     "black", "orange", "rose", "maroon"
])
counter = 0
#for i, ((new_step, orig_step), s) in enumerate(sequences[:200]):
for i, s in enumerate(sequences[:200]):
    objects = tuple(sorted(dict(s).keys()))
    if all(o in our_colors for o in objects):
        print(i, s)
        counter += 1
print(counter)

In [ ]:
objects_to_id ={}
same_objects = []
for i, s in enumerate(task._task.sequences[:200]):
    # print(i, s)
    objects = tuple(sorted(dict(s).keys()))
    # if  "cyan" in objects and not "navy" in objects:
    #     print(i, objects)
        # == ('blue', 'navy', 'cyan'):
    # if len(objects) == 1:
    # if 'green' in objects and 'teal' in objects:
        # print(i, s)
    if objects in objects_to_id:
        print(i, objects_to_id[objects], objects)
        same_objects.append((i, objects_to_id[objects]))
    else:
        objects_to_id[objects] = i

In [ ]:
import json
with open('tower.json') as fid:
    sequences = json.load(fid)

our_colors = set(
    ["yellow", "green", "cyan",
     "blue", "white", 
     "black", "orange", "rose", "red", "lime"
])
counter = 0
for i, s in enumerate(sequences[:200]):
    objects = tuple(sorted(dict(s).keys()))
    if all(o in our_colors for o in objects):
        print(i, s)
        counter += 1
print(counter)

In [ ]:
variations = [('B', 'P', 'G')]
new_variations = variations.copy()
for c in variations[0]:
    print(c, flush=True)
    new_variations += list(itertools.permutations(list(variations[0]) + [c]))
    print(len(new_variations), len(variations), flush=True)
print(new_variations)

In [ ]:
list(itertools.permutations(variations + [c]))

# C2FARM

In [ ]:
import pandas as pd
root_dir = Path('/scratch/jeanzay/scratch/logs/vln-robot/c2farm/10/seed0/')
data_csv = sorted(list(root_dir.glob('*/C2FARM+QTE/seed*/data.csv')))
for data in data_csv:
    task = data.parents[2].name
    method = data.parents[1].name
    res = pd.read_csv(data)
    ret = res['train_envs/return']
    n = ret.shape[0]
    print(f"{task}-{method}", '\ttrain max', ret.max(), '\ttrain mean last 10%', ret[int(n*0.9):].mean())
    # plt.title(f"{task}-{method}")
    # plt.plot(data['eval_envs/return'])
    # plt.show()

# Results

In [ ]:
import pandas as pd
columns = ['taskvar', 'ckpt', 'seed', 'sr', 'test']
df = pd.read_csv('/home/pguhur/Desktop/records-jz.csv', names=columns)
df.drop(columns=['seed', 'test'], inplace=True)

# discard multitask / multivar
multi = df['ckpt'].str.contains('\+100|\+45|multi')
df = df[~multi]

# discard obsolete xp
# zinstr = df['ckpt'].str.contains('zinstr')
# df = df[zinstr]

# keep only 10-tasks
with open('10-tasks.txt') as fid:
    tasks = [f"{t.strip()}-0"  for t in fid.readlines()]
tasks_search  = "|".join(tasks)
print(tasks, tasks_search)
df = df[df['taskvar'].str.contains(tasks_search)]
# for t in tasks:
#     df[t] = df['taskvar'].str.contains(f'{t}-')

df['Visual Tokens'] = np.select([
        df['ckpt'].str.contains('plain'),
        df['ckpt'].str.contains('tnc'),
        df['ckpt'].str.contains('tnhw'),
    ], ['X', 'Channel', 'Patch'])
df['Point Clouds'] = np.where(df['ckpt'].str.contains('pcd'), 'V', 'X')
df['Gripper Position'] = np.where(df['ckpt'].str.contains('gp_attn|gripper'), 'V', 'X')

df['Multiview'] = np.where(df['ckpt'].str.contains('plain'), 'X', 'V')
df['History'] = np.where(df['ckpt'].str.contains('stateless|plain'), 'X', 'V')
df['Attn'] = np.select([
        df['ckpt'].str.contains('plain'),
        df['ckpt'].str.contains('_cm'),
        ~df['ckpt'].str.contains('_cm'),
    ], ['X', 'Cross', 'Self'])

df['Mask Obs'] = np.where(df['ckpt'].str.contains('mask'), 'V', 'X')

# Table
# 	Visual Tokens	Point Clouds	Gripper Position	Multiview	History	Attn	Mask Obs
# R1	X	X	X	X	X	X	X
# R2	Channel	X	X	V	X	Self	X
# R3	Channel	V	X	V	X	Self	X
# R4	Channel	V	V	V	X	Self	X
# R5	Channel	V	V	V	V	Self	X
# R6	Channel	V	V	V	V	Self	V
# R7	Patch	V	V	V	V	Self	V
# R8	Patch	V	V	V	V	Cross	V
df

In [ ]:
criteria = ['Visual Tokens', 'Point Clouds', 'Gripper Position', 'Multiview', 'History', 'Attn', 'Mask Obs']
agg = df.groupby(by=['taskvar'] + criteria).agg({"sr": [np.mean, np.std, np.size]})
table = agg.groupby(by=criteria).agg({('sr', 'mean'): [np.mean], ('sr', 'std'): np.mean, ('sr', 'size'): np.sum})
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
table

In [ ]:
ablation = pd.read_csv('ablation.csv',index_col=0)
ablation

In [ ]:
df.columns

In [ ]:
for row_id, row in ablation.iterrows():
    # masks = [df[key] == value for key, value in row.items()]
    # mask = masks[0]
    # for m in masks[1:]:
    #     mask &= m
    index = int(row_id[1:])
    mask = df['ckpt'].str.contains(f'row{index}')
    
    if index > 5:
        mask &= (
            df['ckpt'].str.contains(f'mask2') |
            df['taskvar'].str.contains(
                f'pick_up_cup|push_button|reach_target|slide_block_to_target|take_umbrella'
            ) 
        )

    print("="*80)
    print(row_id, row)
    
    print('-'*80, '\n', 'Items')
    df_mask = df[mask]
    print(df_mask)
    
    print('-'*80, '\n', 'Agg/tasks')
    agg = df_mask.groupby(by=['taskvar'] + criteria).agg({"sr": [np.mean, np.std, np.size]})
    print(agg)
    
    print('-'*80, '\n', 'Agg/row')
    table = agg.groupby(by=criteria).agg({('sr', 'mean'): [np.mean], ('sr', 'std'): np.mean, ('sr', 'size'): np.sum})
    print(table)

    print("")
    # break

# Versatility Figure

In [ ]:
df_sv = pd.read_csv('/home/pguhur/Desktop/records-singleview.csv', names=columns)
df_sv.drop(columns=['seed', 'test'], inplace=True)
mask = df_sv["ckpt"].str.contains("singleview")
df_sv = df_sv[mask]
df_sv['cam'] = df_sv['ckpt'].apply(lambda x: x.split('singleview-')[1].split('_')[0])
df_sv = df_sv.groupby(by=['taskvar']).agg({"sr": np.max})
df_sv = df_sv.reset_index()

In [ ]:
df_sv.reset_index(inplace=True)

In [ ]:
df_sv.head()

In [ ]:
import pandas as pd
columns = ['taskvar', 'ckpt', 'seed', 'sr', 'test']
df_mv = pd.read_csv('/home/pguhur/Desktop/records2.csv', names=columns, header=1)
df_mv.drop(columns=['seed', 'test'], inplace=True)

df_st = pd.read_csv('/home/pguhur/Desktop/records-jz.csv', names=columns)
df_st.drop(columns=['seed', 'test'], inplace=True)

df_sv = pd.read_csv('/home/pguhur/Desktop/records-singleview.csv', names=columns)
df_sv.drop(columns=['seed', 'test'], inplace=True)
mask = df_sv["ckpt"].str.contains("singleview")
df_sv = df_sv[mask]
df_sv['cam'] = df_sv['ckpt'].apply(lambda x: x.split('singleview-')[1].split('_')[0])
df_sv = df_sv.groupby(by=['taskvar']).agg({"sr": np.max})
df_sv = df_sv.reset_index()

df_al = pd.read_csv('/home/pguhur/Desktop/records-al.csv', names=['task', 'sr', "etc"])
mask = df_al["sr"].str.contains("seed")
df_al.loc[mask, "sr"]  = pd.to_numeric(df_al.loc[mask, "etc"])

task_desc = pd.read_csv("versatility.csv")
# task_desc.set_index("Task")

def found_result(task_name, df):
    sr = df[df['taskvar'].str.contains(task_name) & df['ckpt'].str.contains("tnhw_cm.*mask2")]["sr"]
    if sr.size == 0:
        sr = df[df['taskvar'].str.contains(task_name) & df['ckpt'].str.contains("tnhw_cm.*mask")]["sr"]
    if sr.size == 0:
        sr = df[df['taskvar'].str.contains(task_name) & df['ckpt'].str.contains("tnhw_cm.*")]["sr"]
    if sr.size == 0:
        sr = df[df['taskvar'].str.contains(task_name)]["sr"]  
    return sr.mean()

# add model results.
task_desc['MV'] = task_desc.apply(
    lambda row : found_result(row["Task"], df_mv), axis = 1
)

def found_result(task_name, df):
    sr = df[df['taskvar'].str.contains(task_name) & df['ckpt'].str.contains("tnhw_cm.*mask2")]["sr"]
    if sr.size == 0:
        sr = df[df['taskvar'].str.contains(task_name) & df['ckpt'].str.contains("tnhw_cm.*mask")]["sr"]
    if sr.size == 0:
        sr = df[df['taskvar'].str.contains(task_name) & df['ckpt'].str.contains("tnhw_cm.*")]["sr"]
    if sr.size == 0:
        sr = df[df['taskvar'].str.contains(task_name)]["sr"]  
    return sr.mean()

task_desc['Stateless'] = task_desc.apply(
    lambda row : found_result(row["Task"], df_st), axis = 1
)

def found_result3(task_name, df):
    sr = df[df['taskvar'].str.contains(task_name)]["sr"]  
    return sr.mean()

task_desc['SV'] = task_desc.apply(
    lambda row : found_result3(row["Task"], df_sv), axis = 1
)

def found_result2(task_name, df):
    sr = df[df['task'].str.contains(task_name)]["sr"]
    return pd.to_numeric(sr).mean()

task_desc['AL'] = task_desc.apply(
    lambda row : found_result2(row["Task"], df_al), axis = 1
)



nan = task_desc.isnull().values.any(1)
print(task_desc[nan])
print('# Nan', nan.sum())
print('# Task', (~nan).sum())
task_desc = task_desc[~nan]


In [ ]:
group_by = task_desc.groupby(by=['Set']).agg({
    "MV": [np.mean, np.size], 
    "AL": [np.mean, np.size], 
    "Stateless": [np.mean, np.size],
    "SV": [np.mean, np.size]
})
group_by = group_by.sort_index()
group_by

In [ ]:
tasks = task_desc[task_desc["MV"] > 0.44]["Task"]
tasks.sample(n=50, replace=False).to_csv("50-tasks.csv", header=False, index=False)
tasks.sample(n=30, replace=False).to_csv("30-tasks.csv", header=False, index=False)
task_desc.sample(n=70, replace=False)["Task"].to_csv("70-tasks.csv", header=False, index=False)

In [ ]:
fig = plt.figure(figsize=(10, 4))
methods = ['MV', 'AL']
for i, method in enumerate(methods):
    series = sorted(list(task_desc.groupby(by=['Set'])), key=lambda s: s[1]['MV'].mean(), reverse=False)
    legends = [s[0] for s in series]
    x = [np.asarray(s[1][method]) for s in series]
    ax = plt.subplot(1,2,i+1)
    hist = plt.hist(x,
                    histtype='stepfilled',
                    cumulative=-1,
              bins=100,  stacked=True)
    if i == len(methods) - 1:
        plt.legend(legends, loc='upper right')
    plt.xlabel('Threshold as % of success rate')
    plt.ylabel('Total number of tasks above this threshold')
    plt.xlim([0,1])

# handles, labels = ax.get_legend_handles_labels()
# fig.legend(handles, labels, loc='upper center')
plt.savefig(f'cum-sr.pdf')
plt.show()


# Convert tower annotations

In [ ]:
with open("/home/pguhur/Downloads/amt-train.fixed.json") as fid:
    data = json.load(fid)

data[0]

In [ ]:
item = data[0]
colors = [item["color"][1]] + item["color"][:-1:2]
variations = 
print(color)

In [ ]:
items = [ {
    "id": i,
    "createdTime": "2020-01-01T00:00:00",
    "fields": {
    "instruction": item['sentence'],
    "task": "tower3",
    "variation": get_variation(item)
    }
} for item in data]

# Checking on embeddings

In [ ]:
import pickle
with open('instructions-clip.pkl', 'rb') as fid:
    embeddings = pickle.load(fid)

In [ ]:
var = [65, 53]
for v in var:
    print(task._task.sequences[v])

In [ ]:
pbs = [embeddings['push_buttons'][v] for v in var]

In [ ]:
means = [p[0].mean(0) for p in pbs]
plt.figure(figsize=(10, 20))

plt.subplot(311)
plt.imshow(means[0].view(16,32))
plt.title('Combaring avg embeddings for 2 variations w/ the same objects')
plt.colorbar()
plt.subplot(312)
plt.imshow(means[1].view(16,32))
plt.colorbar()
plt.subplot(313)
plt.imshow((means[1]-means[0]).view(16,32))
plt.colorbar()
plt.show()

In [ ]:
means = [p[0].mean(0) for p in pbs]
plt.figure(figsize=(10, 20))
plt.subplot(311)
plt.imshow(means[0].view(16,32))
plt.title('Combaring avg embeddings for 2 tasks')
plt.colorbar()
plt.subplot(312)
e2 = embeddings['tower3'][0][0].mean(0).view(16,32)
plt.imshow(e2)
plt.colorbar()
plt.subplot(313)
plt.imshow(means[1].view(16,32) - e2)
plt.colorbar()
plt.show()

In [ ]:
means = [p[0].mean(0) for p in pbs]
plt.figure(figsize=(10, 20))
plt.subplot(311)
plt.imshow(means[0].view(16,32))
plt.title('Combaring avg embeddings for 2 variations with different objects')
plt.colorbar()
plt.subplot(312)
e2 = embeddings['push_buttons'][0][0].mean(0).view(16,32)
plt.imshow(e2)
plt.colorbar()
plt.subplot(313)
plt.imshow(means[1].view(16,32) - e2)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(embeddings['tower3'][0][0].mean(0).view(16,32))

In [ ]:
plt.imshow(embeddings['push_buttons'][0][0,30].view(16,32))
plt.colorbar()

In [ ]:
embeddings['push_buttons'].keys()

In [ ]:
embeddings['push_buttons'][65][0].mean(0).shape

In [ ]:
F.cosine_similarity(
             embeddings['push_buttons'][65][0].mean(0),
             embeddings['push_buttons'][53][0].mean(0),
            dim=0
        ).mean()

In [ ]:
from torch.nn import functional as F
import itertools
print('same task, same objects')
cossims = []
for var1, var2 in same_objects:
    # if var1 in embeddings['push_buttons'] and var2 in embeddings['push_buttons']
    cossim = F.cosine_similarity(
             embeddings['push_buttons'][var1][:2].mean(1),
             embeddings['push_buttons'][var2][:2].mean(1),
            dim=1
        ).max()
        # 
    cossims.append(cossim)
print('\t',torch.Tensor(cossims).mean())


print('same task, different objects')
cossims = []
for var1, var2 in itertools.product(range(100), range(100)):
    if (var1, var2) in same_objects or var1 == var2:
        continue
    cossim = F.cosine_similarity(
             embeddings['push_buttons'][var1][:2].mean(1),
             embeddings['push_buttons'][var2][:2].mean(1),
            dim=1
        ).max()
        # 
    cossims.append(cossim)
print('\t',torch.Tensor(cossims).mean())

In [ ]:
print('different task')
cossims = []
for task1, task2 in itertools.product(embeddings.keys(), embeddings.keys()):
    if task1 == task2:
        continue
    cossim = F.cosine_similarity(
             embeddings[task1][0][:2].mean(1),
             embeddings[task2][0][:2].mean(1),
            dim=1
        ).max()
        # 
    cossims.append(cossim)
print('\t',torch.Tensor(cossims).mean())

# Visualize prediction

In [ ]:
from multi_task_baselines import get_model, collate_fn, Arguments as TrainArguments

dataset = "/scratch/jeanzay/scratch/datasets/real-dataset-0/"
sys.argv = [
    'foo', '--dataset', str(dataset), 
    '--tasks', 'push_buttons'
]

args = TrainArguments().parse_args()
args.checkpoint = "xp-jz/hf-s2r+noise_push_buttons/version521949/model.step=90-value=0.pth"
args.tr_token = "tnhw_cm"
args.pcd_token = 'none' 
args.gripper_pose = "none"
args.z_mode = "instr2"
args.temp_len = 0
args.instructions = "instructions-clip.pkl"
optimizer, meta_model, loss_and_metrics = get_model(args)
model = meta_model["model"]
t_dict = meta_model["t"]
z_dict = meta_model["z"]

In [ ]:
dataset = Path("/scratch/cooper/datasets/hiveformer/s2r-dr-dataset-0/")
# dataset = Path("/scratch/cooper/datasets/hiveformer/real-dataset-0/")
taskvar = [('push_buttons', 0)]
instruction = load_instructions("instructions-clip.pkl")
dataset = RLBench(
    root=dataset,
    taskvar=taskvar,
    instructions=instruction,
    gripper_pose=args.gripper_pose,
    taskvar_token=False,
    max_episode_length=6,
    max_episodes_per_taskvar=100,
    cache_size=0,
    jitter=True,
    training=True,
    cameras=('wrist', 'left_shoulder', 'right_shoulder'),
)
loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn,
)

In [ ]:
iter_loader = iter(loader)
sample = next(iter_loader)

In [ ]:
sample = next(iter_loader)

In [ ]:
device = t_dict[list(t_dict.keys())[0]].device

rgbs = sample["rgbs"].to(device)
pcds = sample["pcds"].to(device)
gripper = sample["gripper"].to(device)
outputs = sample["action"].to(device)
padding_mask = sample["padding_mask"].to(device)

instr = sample["instr"]
if instr is not None:
    instr = instr.to(device)

taskvar = sample["taskvar"]
if taskvar is not None:
    taskvar = taskvar.to(device)

frame_id = sample["frame_id"]
tasks = sample["task"]
t = torch.stack([t_dict[task][fid] for task, fid in zip(tasks, frame_id)])
z = torch.stack([z_dict[task][fid] for task, fid in zip(tasks, frame_id)])

model.eval()
with torch.no_grad():
    pred = model(
        rgbs,
        pcds,
        padding_mask,
        t,
        z,
        instr,
        gripper,
        taskvar,
    )

In [ ]:
rgbs = (sample["rgbs"][0].cpu() + 1) * 0.5
rgbs = rgbs.permute(0, 1, 3, 4, 2)

for step_id, (attns, rgb) in enumerate(zip(pred['attention'], rgbs)):
    for cam_id, (a, r) in enumerate(zip(attns[1:], rgb[1:3])):
        i, j = (a[0] == a[0].max()).nonzero()[0].cpu()
        
        plt.subplot(2, 2, cam_id + 1)
        plt.imshow(r)
        plt.scatter(j,i, color="r", marker="+")
        
        plt.subplot(2, 2, cam_id + 3)
        plt.imshow(a[0].detach().cpu().log())
        plt.scatter(j,i, color="r", marker="+")
            
    plt.show()

# Increase difficulty of the task PushButtons

In [ ]:
import itertools
import matplotlib.pyplot as plt
import torch
from utils import (
    load_instructions
)
from create_data import RLBench

taskvar = list(itertools.product(['push_buttons'], [0]))
instruction = load_instructions("instructions-clip.pkl")
max_episode_length = 6



In [ ]:
frame_ids = sample[0].copy()
state_ls = sample[1].copy()
action_ls = sample[2].copy()
attn_indices = sample[3].copy()
gripper_pos = sample[4].copy()

orig_step, new_step = new_variations[0][0]
frame_ids += [len(frame_ids), len(frame_ids) + 1]
state_ls.insert(1 + 2 * new_step, sample[1][1 + 2 * orig_step])
state_ls.insert(2 + 2 * new_step, sample[1][2 + 2 * orig_step])
action_ls.insert(1 + 2 * new_step, sample[2][1 + 2 * orig_step])
action_ls.insert(2 + 2 * new_step, sample[2][2 + 2 * orig_step])
attn_indices.insert(1 + 2 * new_step, sample[3][1 + 2 * orig_step])
attn_indices.insert(2 + 2 * new_step, sample[3][2 + 2 * orig_step])
gripper_pos.insert(1 + 2 * new_step, sample[4][1 + 2 * orig_step])
gripper_pos.insert(2 + 2 * new_step, sample[4][2 + 2 * orig_step])

In [ ]:
states = torch.stack(state_ls)
rgbs = states[:, :, 0]
rgbs = (rgbs + 1) *0.5
rgbs = rgbs.permute(0, 1, 3, 4, 2)

for i in range(rgbs.shape[0]):
    plt.imshow(rgbs[i, 0])
    # plt.title(("{:.2f} "*8).format(*sample['action'][i, :8].tolist()))
    plt.show()

# Wav2vec2

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

def record_mic(duration: float, fs: int) -> np.ndarray:
    return sd.rec(duration * fs, samplerate=fs, channels=1, dtype='float64')

class MicToText:
    def __init__(self, duration: float = 5., sampling_rate: int = 44100, device: str = 'cpu'):
        self.processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        self.model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
        self.model = self.model.to(device).eval()
        self.duration = duration
        self.sampling_rate = sampling_rate

    def __call__(self) -> str:
        record = record_mic(self.duration, self.sampling_rate)
        
        # audio file is decoded on the fly
        inputs = self.processor(record, sampling_rate=self.sampling_rate, return_tensors="pt")
        with torch.no_grad():
            logits = self.model(**inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # transcribe speech
        transcription = processor.batch_decode(predicted_ids)
        return transcription[0]
    
class TextToToken:
    def __init__(self, model_max_length: int = 42, device: str = 'cpu'):
        self.model = transformers.CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
        self.model = self.model.to(device)
        self.device = device
        self.model_max_length = model_max_length
        self.tokenizer = transformers.CLIPTokenizer.from_pretrained(
            "openai/clip-vit-base-patch32"
        )
        self.tokenizer.model_max_length = model_max_length
    
    def __call__(self, instruction: str) -> torch.Tensor:
        tokens = tokenizer(instruction, padding="max_length")["input_ids"]
        lengths = [len(t) for t in tokens]
        if any(l > self.model_max_length for l in lengths):
            raise RuntimeError(f"Too long instructions: {lengths}")

        tokens = torch.tensor(tokens).to(self.device)
        with torch.no_grad():
                pred = model(tokens).last_hidden_state
            
        return pred

# Instructions Push Repeated Buttons

In [ ]:
with open('push_repeated_buttons.json') as fid:
    variations = json.load(fid)
variations[124]

In [ ]:
from convert_push_buttons import generate_repeated_buttons_instructions

In [ ]:
generate_repeated_buttons_instructions(['rose', 'orange', 'black'])

In [ ]:
with open('annotations2.json') as fid:
    data = json.load(fid)
    
with open('push_repeated_buttons.json') as fid:
    variations = json.load(fid)
variations = {
    i: [c for c, rgb in s] for i, (_, s) in enumerate(variations)
}

In [ ]:
data2 = []

for item in data:
    var = item["fields"]["variation"]
    colors = variations[var]
    if len(colors) != len(set(colors)):
        item["fields"]["instr_old"] = item["fields"]["instruction"]
        item["fields"]["instruction"] = ""
    data2.append(item)
    
with open('annotations2.json', "w") as fid:
    json.dump(data,fid,indent=2)

In [ ]:
with open('annotations2.json') as fid:
    data = json.load(fid)
    
for item in data:
    if item["fields"]["instruction"] == "":
        var = item["fields"]["variation"]
        colors = variations[var]
        print(var, colors)
        print(item["fields"]["instr_old"])
        item["fields"]["instruction"] = input(">")
        
        with open('annotations2.json', "w") as fid:
            json.dump(data,fid, indent=2)
            
with open('annotations2.json', "w") as fid:
            json.dump(data,fid, indent=2)

# Silva et al

### Obtaining pose & velocity of each object

In [ ]:
import os
import sys
from os.path import join, dirname, abspath, isfile

# sys.path.insert(0, join(CURRENT_DIR, '..'))  # Use local RLBench rather than installed

import traceback
import readline

from pyrep.const import RenderMode

from rlbench.backend import task
from rlbench.backend.const import TTT_FILE
from pyrep import PyRep
from pyrep.robots.arms.panda import Panda
from pyrep.objects.shape import Shape
from pyrep.objects.proximity_sensor import ProximitySensor
from pyrep.objects.dummy import Dummy
from pyrep.robots.end_effectors.panda_gripper import PandaGripper
from rlbench.backend.scene import Scene
from rlbench.backend.exceptions import *
from rlbench.observation_config import ObservationConfig, CameraConfig
from rlbench.backend.robot import Robot
from rlbench.utils import name_to_task_class
# from task_validator import task_smoke, TaskValidationError
import shutil

pr = PyRep()
ttt_file = join('/home/pguhur/src/vln-robot/RLBench/rlbench', TTT_FILE)
pr.launch(ttt_file, responsive_ui=True)
pr.step_ui()

robot = Robot(Panda(), PandaGripper())
cam_config = CameraConfig(rgb=True, depth=False, mask=False,
                          render_mode=RenderMode.OPENGL)
obs_config = ObservationConfig()
obs_config.set_all(False)
obs_config.right_shoulder_camera = cam_config
obs_config.left_shoulder_camera = cam_config
obs_config.overhead_camera = cam_config
obs_config.wrist_camera = cam_config
obs_config.front_camera = cam_config

scene = Scene(pr, robot, obs_config)

In [ ]:
import pickle
with open('/tmp/rlbench_data/push_button/variation0/episodes/episode0/low_dim_obs.pkl', 'rb') as fid:
    low_dim_obs = pickle.load(fid)

In [ ]:
low_dim_obs[3].misc['state']

In [ ]:
import torch
import einops
import numpy as np
from torch import nn
glove_path = '/scratch/diffrac/glove.6B.50d.txt'

embeddings_dict = {}
with open(glove_path, 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], "float32")
        embeddings_dict[word] = torch.Tensor(vector)

In [ ]:
instr = 'Reach the blue ball'

embeddings = torch.stack([embeddings_dict[w] for w in instr.lower().split()])
embeddings = embeddings.unsqueeze(0)
print(embeddings.shape)

lstm = nn.LSTM(input_size=50, hidden_size=30, bidirectional=True, batch_first=True)
output, (hn, cn) = lstm(embeddings)

hn = einops.rearrange(hn, 'dn b h -> b (dn h)')
print(hn.shape)

In [ ]:
import pickle as pkl

file = '/scratch/jeanzay/scratch/datasets/vlmbench-raw/train/stack_cubes_color/variation3/episodes/episode0/configs.pkl'
with open(file, 'rb') as fid:
    data = pkl.load(fid)

In [ ]:
data.__dict__.keys()

In [ ]:
data.high_level_descriptions

In [ ]:
data.task_attributes

# VLMBench

In [ ]:
from amsolver.backend.scene import Scene
from amsolver.backend.robot import Robot
from amsolver.utils import name_to_task_class
from amsolver.backend.const import TTT_FILE
from amsolver.observation_config import ObservationConfig, CameraConfig
from pyrep.robots.end_effectors.panda_gripper import PandaGripper
from pyrep.robots.arms.panda import Panda
from pyrep import PyRep
import os
from pathlib import Path

In [ ]:
pr = PyRep()
ttt_file = os.path.join('/home/pguhur/src/vln-robot/vlmbench/amsolver', TTT_FILE)
pr.launch(ttt_file, responsive_ui=True)
pr.step_ui()

robot = Robot(Panda(), PandaGripper())
obs_config = ObservationConfig()
obs_config.set_all(False)

scene = Scene(pr, robot, obs_config)

In [ ]:
tasks = [t.stem for t in Path('/home/pguhur/src/vln-robot/vlmbench/vlm/tasks/').glob('*.py')]

root_dir = '/gpfsstore/rech/vuo/uok79zh/datasets/vlmbench/valid/unseen'

tarfiles = {
    'wipe_table': f'{root_dir}/wipe.tar.gz',
    'drop_pen': f'{root_dir}/drop.tar.gz',
    'stack_cubes': f'{root_dir}/stack.tar.gz',
    'place_into_shape_sorter': f'{root_dir}/shape_sorter.tar.gz',
    'pour_demo': f'{root_dir}/pour.tar.gz',
    'open_door': f'{root_dir}/door.tar.gz',
    'open_drawer': f'{root_dir}/drawer.tar.gz',
    'pick_cube': f'{root_dir}/pick.tar.gz',
}
for task_name in tasks[1:]:
    task_class = name_to_task_class(task_name,parent_folder='vlm')
    task = task_class(pr, robot)
    # scene.load(task, ttms_folder='./vlm/task_ttms')
    task_stem = '_'.join(task_name.split('_')[:-1])
    
    if task_stem not in tarfiles:
        # print(task_stem)
        continue
        
    tarfile = tarfiles[task_stem]
    
    try:
        var_count = task.variation_count()
    except NotImplementedError:
        continue
        
    for i in range(var_count):
        print(f"{task_name},{i},{tarfile}")


### VLMbench: keypoint detector

In [ ]:
* use the front camera

In [ ]:
env.obs_config.front_camera.mask = False

In [ ]:
demos = env.get_demo('drop_pen_color', 16, 1)

In [ ]:
env.apply_cameras

In [ ]:
%cd ../vlmbaseline

import sys

from utils import RLBenchEnv
env = RLBenchEnv(
            data_path='/scratch/jeanzay/scratch/datasets/vlmbench-raw/valid-seen',
            apply_rgb=True,
            apply_pc=True,
            apply_cameras=("front"),
        )

# Why is it not done from the constructor?
env.obs_config.front_camera.rgb = True

In [ ]:
demos = env.get_demo('drop_pen_color', 16, 1)
demo = demos[0]

from utils import _is_stopped

def keypoint_discovery(demo):
    episode_keypoints = []
    prev_gripper_open = demo[0].gripper_open
    stopped_buffer = 0
    for i, obs in enumerate(demo):
        stopped = _is_stopped(demo, i, obs, stopped_buffer)
        stopped_buffer = 4 if stopped else stopped_buffer - 1
        # If change in gripper, or end of episode.
        last = i == (len(demo) - 1)
        if i > 5 and (obs.gripper_open != prev_gripper_open or last or stopped):
            episode_keypoints.append(i)
        prev_gripper_open = obs.gripper_open
    if (
        len(episode_keypoints) > 1
        and (episode_keypoints[-1] - 1) == episode_keypoints[-2]
    ):
        episode_keypoints.pop(-2)

    return episode_keypoints

keypoint_discovery(demo)

In [ ]:
for k in demo[0].__dict__.keys():
    if getattr(demo[0], k) is not None:
        print(k)


In [ ]:
import matplotlib.pyplot as plt
for k in keypoint_discovery(demo):
    print(k)
    # plt.imshow(
    plt.imshow(demo[k].front_rgb)
    plt.show()